In [32]:
from pyspark.sql.types import StructField , StructType , StringType, IntegerType
from pyspark.sql.functions import col , rand , when
from pyspark.sql import SparkSession , Row , SQLContext
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [16]:
# spark  = SparkSession.builder.master('local[*]').appName('test').config().getorCreate()
people = Row("Id" ,"Name", "Age", "Area of Interest")
p1 = people("1" ,"jack", 22, "Data Science")
p2 = people("2" ,"Leo" ,21 ,"Data Analytics")
p3 = people("3" ,"Luke", 24, "Microservices")
p4 = people("4" ,"Mark" ,21 ,"Data Analytics")
data = [p1,p2,p3,p4]

df = spark.createDataFrame(data)
df.show()

ageDescribe = df.describe('Age')
ageDescribe.show()

ageDescribe = ageDescribe.withColumn('Age',col('Age').cast(StringType()))
ageDescribe = ageDescribe.withColumn('Age',when(ageDescribe['summary'] == 'stddev',1.414).otherwise(ageDescribe['Age']))
ageDescribe.show()

+---+----+---+----------------+
| Id|Name|Age|Area of Interest|
+---+----+---+----------------+
|  1|jack| 22|    Data Science|
|  2| Leo| 21|  Data Analytics|
|  3|Luke| 24|   Microservices|
|  4|Mark| 21|  Data Analytics|
+---+----+---+----------------+

+-------+-----------------+
|summary|              Age|
+-------+-----------------+
|  count|                4|
|   mean|             22.0|
| stddev|1.414213562373095|
|    min|               21|
|    max|               24|
+-------+-----------------+

+-------+-----+
|summary|  Age|
+-------+-----+
|  count|    4|
|   mean| 22.0|
| stddev|1.414|
|    min|   21|
|    max|   24|
+-------+-----+



In [29]:
# df = SQLContext(sc).range(0,10).withColumn('rand1',rand(seed = 10)).withColumn('rand2',rand(seed = 22))
df = SparkSession.builder.getOrCreate().range(0,10).withColumn('rand1',rand(seed = 10)).withColumn('rand2',rand(seed = 22))
df.show()

+---+-------------------+-------------------+
| id|              rand1|              rand2|
+---+-------------------+-------------------+
|  0|0.03422639313807285| 0.9388044512577215|
|  1| 0.3654625958161396| 0.3943255396952755|
|  2| 0.4175019040792016|0.23260361399084917|
|  3|0.16452185994603707|0.02339183228862929|
|  4|0.18141810315190554| 0.8894415403143504|
|  5| 0.9697474945375325| 0.5956629641640896|
|  6|0.34084319330900115| 0.2762195585886885|
|  7| 0.4725977369833597| 0.8378244793454831|
|  8| 0.5996723933366402| 0.6340859398860077|
|  9| 0.6396141227834357|0.49326717925251506|
+---+-------------------+-------------------+



In [30]:
df_cov =  df.cov('rand1','rand2')
df_corr =  df.corr('rand1','rand2')

dataType = [("cov",df_cov),("corr",df_corr)]
schemaType = StructType([StructField('stats',StringType(),True),StructField('Values',StringType(),True)])
df_ = spark.createDataFrame(data = dataType , schema = schemaType )
df_.show()

+-----+--------------------+
|stats|              Values|
+-----+--------------------+
|  cov|-0.00253982823215...|
| corr|-0.03065022425988...|
+-----+--------------------+

